## Szegmentálás és tokenizálás tesztelni egy példa fasta file-on, ami a ESKAPE pathogenekből szárzmazik, ez lehet a példa kód a későbbi feladatokra.


In [126]:
%load_ext autoreload
%autoreload 2
from transformers import AutoTokenizer, AutoModel, utils
#from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings
import yaml
import pathlib
from os.path import join
import os
import sys
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

current_path = str(pathlib.Path(os.getcwd()).parent)
sys.path.append(current_path)

#import sys
#sys.path.append('../)

from config_utils import *
from sequtils import *

%load_ext autoreload
%autoreload 2

defconfig = SeqConfig()
segmentation_params = defconfig.get_segmentation_parameters()         
    
samplefasta_file = '/home/ligeti/github/prokbert/src/prokbert/data/sample_data/ESKAPE_sample.fasta'
contigs = load_contigs(samplefasta_file,IsAddHeader=True,AsDataFrame=True, adding_reverse_complement=False)
contigs['sequence_id'] = [0, 1, 2]
sequences_ls = list(contigs['sequence'])

act_segments = segment_sequence_contiguous(sequences_ls[0], segmentation_params, 1)

            

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
Loading sequence data into memory!
Since the fasta_files_list is a string, not list, we convert to a list.


In [127]:
n_sequence


['TAGAAATGTCCGCGACCTTTCATACATACCACCGGTACGCCCTGGAGATG',
 'ATAATGCTAAATCGTAACCCCACTGCTTAAATGAGCCTTCTGTAAATTTC',
 'GTGACCGGGGTCAGGTTCTCGGCGGCGGCGCGCATCACGTGCTTGCCGAC']

In [128]:
n_sequence = [ str[0:50] for str in contigs['sequence']]
test_contigs =contigs [ ['sequence_id', 'sequence']]
test_contigs['sequence'] = n_sequence
test_contigs.to_json()


/tmp/ipykernel_2461564/3597721582.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_contigs['sequence'] = n_sequence


'{"sequence_id":{"0":0,"1":1,"2":2},"sequence":{"0":"TAGAAATGTCCGCGACCTTTCATACATACCACCGGTACGCCCTGGAGATG","1":"ATAATGCTAAATCGTAACCCCACTGCTTAAATGAGCCTTCTGTAAATTTC","2":"GTGACCGGGGTCAGGTTCTCGGCGGCGGCGCGCATCACGTGCTTGCCGAC"}}'

In [68]:
segmentation_params


{'type': 'contiguous', 'min_length': 0, 'max_length': 512, 'coverage': 1.0}

In [109]:
segment_sequences(contigs, segmentation_params, AsDataFrame = True)
 


Sequences is a list, therefore adding tracking information.
Checking input DataFrame!
Checking input sequence_id is valid primary key in the DataFrame
Creating a DataFrame from the segments. 


,segment_id,sequence_id,segment_start,segment_end,segment
0,0,0,0,512,TAGAAATGTCCGCGACCTTTCATACATACCACCGGTACGCCCTGGA...
1,1,0,512,1024,CTATGAAAAAACATCTTTTAACTCTGACACTTTCCTCTATATTAGC...
2,2,0,1024,1536,TGGGTTGCAGGTTACAACTTTATGCTGGGCAGCGAGAAATTCACTC...
3,3,0,1536,2048,GCCTGCGGCTACCATCGGGAACAGCGTCGCCGGATGTCCAATCTCC...
4,4,0,2048,2560,GCTGCTGCCCTGCATCTGTAAGCACCACTTCACGCGTGGTTCTGTC...
5,5,0,2560,2775,CGTGCTGTTTCCGATTCTGGGTGTACTGTTTGCCTGGTGGAAACCG...
6,6,1,0,512,ATAATGCTAAATCGTAACCCCACTGCTTAAATGAGCCTTCTGTAAA...
7,7,1,512,1024,GTTGTATCAAATGCTTTTTGGCCAGCTAGCACTTCTTTCCATTCAA...
8,8,1,1024,1536,TTCAAATAATTCTTCACGACCAGCGTCTTTCGTAATTTGACGGCTC...
9,9,1,1536,2048,CATAACGATTTTCATCTGATTCATTTTCAGCATCAGGATCGAAATG...


## Adding the coverage based segmentation


1. lépés valamilyen dataframe-t létrehozni, amiben
cumsum, contig_lengths, sequences, contigs, ilyesmi. De nem tudom ezek között mi is a különbség



In [133]:
params['type'] = 'random'
results = segment_sequences(contigs, params, AsDataFrame=True)
results = segment_sequences(contigs, params, AsDataFrame=False)

#pd.DataFrame(results)

results = segment_sequences(contigs, params, AsDataFrame=True)
results

Sequences is a list, therefore adding tracking information.
Checking input DataFrame!
Checking input sequence_id is valid primary key in the DataFrame
Sampling 19 segments from 3 sequences.
Creating a DataFrame from the segments. 
Sequences is a list, therefore adding tracking information.
Checking input DataFrame!
Checking input sequence_id is valid primary key in the DataFrame
Sampling 19 segments from 3 sequences.
Sequences is a list, therefore adding tracking information.
Checking input DataFrame!
Checking input sequence_id is valid primary key in the DataFrame
Sampling 19 segments from 3 sequences.
Creating a DataFrame from the segments. 


,segment_id,sequence_id,segment_start,segment_end,segment
0,0,0,1442,1954,ACTGGCGTTCAACAACCGGCGTAATGCGCTTTACCACCAGTGGGCT...
1,1,0,1479,1991,CAGTGGGCTACCTTCAGGCAATGGTAGCGCCAGTGCGGGAAGAATA...
2,2,0,1821,2333,TCCCTGGCGAATACTTTCCATTACCCACTGCTGCGGACGATCGTGC...
3,3,0,1863,2375,GTGCAGGACAAACTGAATATCTGGATAGCGGCGATGGCTTTCAGCA...
4,4,1,268,780,AGCATAAATGTCTTCAGTATTTTCACGGAAAATAACCATATCAACA...
5,5,1,600,1112,TAACTCGGCTTGCTGCCTTCCAAATATCCGGTCCAATTCCATCACC...
6,6,1,674,1186,GGTACGTTTAATCCTTCAGTTCCTTGAGTAATTTTTTCTGCAGTCA...
7,7,1,704,1216,ATTTTTTCTGCAGTCATAGTTATTTTACCTCCAAATTTTACATCTT...
8,8,1,964,1476,ATCAACATTAGGAATTAATCCTTTTTCTTCTGCCATACGTTTTTCC...
9,9,1,1746,2258,ATAGGAAAATAACTTCTTCAAATTGCGCATTTTCAGCTAGATCATC...


In [102]:

sequence_data = contigs.copy()
seq_lengths = sequence_data.apply(lambda x: len(x['sequence']), axis=1)
sequence_data['seq_lengths'] = seq_lengths
sequence_data['lenght_cum_sum'] = sequence_data['seq_lengths'].cumsum()
Lseqs = sum(seq_lengths)

# The number of segments to be sampled to meet the expected coverage paramteres. Note that the it might be biased if many 'short' - compared the segment sizes, sequences are in the dataset
N_segments = int(np.ceil(segmentation_params['coverage']*Lseqs/segmentation_params['max_length']))


params = segmentation_params




In [101]:

cum_sums = np.int64(sequence_data['lenght_cum_sum'])
sequence_lenghts = np.int64(contig_data['seq_lengths'])
sequences = list(contig_data['sequence'])
sequence_ids = list(sequence_data['sequence_id'])

start_coords = list(np.sort(np.int64(np.random.uniform(0, max(cum_sums), N_segments))))
segmentdb = []

min_segment_length = params['min_length']


for sid, act_sampling_coord in enumerate(start_coords):

    diff = act_sampling_coord-cum_sums

    for i in range(cum_sums.shape[0]):
        if diff[i]<0:
            break
    # Az i. koordináta lesz a listában az a contig, amelyikhez az adott elem tartozik.
    act_sequence_id = sequence_ids[i]
    rel_coord = act_sampling_coord - cum_sums[i] + sequence_lenghts[i]
    segment_end = min(rel_coord + Ls, sequence_lenghts[i])
    expected_segment_length = segment_end - rel_coord
    if expected_segment_length < 2:
        print('Too short segment, skip!')
        continue
    new_segment = sequences[i][rel_coord:segment_end]
    new_id = f'{sid}'

    new_record = {'segment_id' : sid,
                 'sequence_id': act_sequence_id,
                 'segment_start' : rel_coord,
                 'segment_end' : segment_end,
                 'segment' : new_segment}
    segmentdb.append(new_record)
segmentdb = pd.DataFrame(segmentdb)


KeyError: 0